In [ ]:
!pip install -q crewai langchain-openai openai python-dotenv gradio


In [ ]:
import os
from google.colab import userdata
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI
import gradio as gr



In [ ]:
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

llm = ChatOpenAI(model="gpt-4o", api_key=os.environ["OPENAI_API_KEY"])


In [ ]:



# Define Agents
mood_analyzer = Agent(
    role="Mood Analyzer",
    goal="Understand the user's emotional state",
    backstory="You are skilled in emotional intelligence and psychological analysis. You interpret emotions from text input.",
    llm=llm
)

companion = Agent(
    role="Companion",
    goal="Offer emotional support",
    backstory="You are a friendly and caring virtual companion who can provide supportive responses to users based on their emotional state.",
    llm=llm
)

selfcare_recommender = Agent(
    role="Self-Care Recommender",
    goal="Suggest self-care activities",
    backstory="You suggest actionable self-care tips for users based on their current mood and situation.",
    llm=llm
)

# Tasks — use task variables to fetch results later
task_mood = Task(
    description="Analyze this message and return the user's emotional state in ONE WORD:\n\n'{user_input}'",
    expected_output="One word representing the user's emotion.",
    agent=mood_analyzer
)

task_companion = Task(
    description="Respond empathetically to this message:\n\n'{user_input}'",
    expected_output="A short paragraph that shows understanding and emotional support.",
    agent=companion
)

task_selfcare = Task(
    description="Recommend 2-3 practical self-care activities based on this message:\n\n'{user_input}'",
    expected_output="Bullet points of actionable self-care tips.",
    agent=selfcare_recommender
)


In [ ]:
def run_mental_health_crew(user_input):
    task_mood.description = task_mood.description.format(user_input=user_input)
    task_companion.description = task_companion.description.format(user_input=user_input)
    task_selfcare.description = task_selfcare.description.format(user_input=user_input)

    crew = Crew(
        agents=[mood_analyzer, companion, selfcare_recommender],
        tasks=[task_mood, task_companion, task_selfcare],
        verbose=True
    )

    crew.kickoff()

    return (
        task_mood.output,
        task_companion.output,
        task_selfcare.output
    )


In [ ]:
# def handle_user_input(user_text):
#     if not user_text.strip():
#         return "Please write how you're feeling.", "", ""

#     results, task_mood, task_companion, task_selfcare = run_mental_health_crew(user_text)

#     return (
#         results[task_mood].output,
#         results[task_companion].output,
#         results[task_selfcare].output
#     )


In [ ]:
import gradio as gr

def handle_user_input(user_text):
    if not user_text.strip():
        return "Please describe how you're feeling.", "", ""
    mood, response, selfcare = run_mental_health_crew(user_text)
    return mood, response, selfcare

gr.Interface(
    fn=handle_user_input,
    inputs=gr.Textbox(label="How are you feeling today?"),
    outputs=[
        gr.Textbox(label="Detected Mood"),
        gr.Textbox(label="Supportive Message"),
        gr.Textbox(label="Self-Care Tips")
    ],
    title="Mental Health Companion",
    theme="soft"
).launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2a16a90de8ca843d6a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
